# HW 00: Perceptron

# Regularization Techniques: L1 and L2 Regularization

Regularization is a crucial concept in machine learning, ensuring that models generalize well to new, unseen data. Among the commonly used regularization methods are L1 and L2 regularizations. This document outlines their concepts, applications, and key differences.

## L2 Regularization (Ridge Regularization)

L2 regularization, also known as Ridge or Tikhonov regularization, combats overfitting, particularly prevalent in linear regression and neural networks.

### Key Features of L2 Regularization:

- **Penalizing Large Coefficients:**
  L2 regularization penalizes the model for having large weights. This discourages over-reliance on any single feature and thereby guards against overfitting.

- **Mathematical Formulation:**
  L2 regularization augments the original loss function by adding the squared magnitude of the weights. In mathematical terms:
  $$
  L_{regularized} = L + \alpha \sum_i w_i^2
  $$
  Where:
  - $ L_{regularized} $ is the regularized loss.
  - $ \alpha $ is the regularization strength.
  - $ w_i $ represents each weight in the model.

### Benefits:

- Prevents overfitting by constraining the magnitude of the weights.
- Leads to more stable models, especially when features are correlated.
- Encourages smaller weight values, making potentially smoother models.

## L1 Regularization (Lasso Regularization)

L1 regularization, or Lasso regularization, is another technique that prevents overfitting. Unlike L2, which penalizes the square of the weights, L1 penalizes the absolute value of the weights.

### Mathematical Formulation:
L1 regularization is mathematically expressed as:
$$
L_{regularized} = L + \alpha \sum_i |w_i|
$$

### Key Difference:

- While L2 regularization tends to make weights small but non-zero, L1 can make some weights exactly zero, leading to sparse models.


### Hint:

You get the model weights by using

```python
p in model.parameters
```

## Q1.1: Implement L1 and L2 regularization and explain the effect of regularization. 

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.datasets import make_moons, make_blobs


np.random.seed(1337)
random.seed(1337)

# make up a dataset
X, y = make_moons(n_samples=100, noise=0.1)
y = y*2 - 1  # make y be -1 or 1

# visualize in 2D
plt.figure(figsize=(5,5))
plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')

# Model definition in PyTorch using nn.Sequential
## Create a PyTorch NN with two layers and 16 neurons in each layer
print(model)

# loss function
def loss(batch_size=None):
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]

    inputs = torch.tensor(Xb, dtype=torch.float32)
    labels = torch.tensor(yb, dtype=torch.float32).view(-1, 1)

    # forward the model to get scores
    scores = model(inputs)

    # svm "max-margin" loss
    losses = F.relu(1 + -labels*scores)
    data_loss = losses.mean()

    # Loss
    ### Implement L1 and L2 regularization here
    total_loss = data_loss

    # also get accuracy
    accuracy = ((labels > 0) == (scores > 0)).float().mean()
    return total_loss, accuracy

total_loss, acc = loss()
print(total_loss.item(), acc.item())

# optimization
for k in range(100):
    # forward
    total_loss, acc = loss()

    # backward
    model.zero_grad()
    total_loss.backward()

    # update (sgd)
    learning_rate = 1.0
    with torch.no_grad():
        for p in model.parameters():
            p -= learning_rate * p.grad

    if k % 10 == 0:
        print(f"step {k} loss {total_loss.item()}, accuracy {acc*100}%")


# visualize decision boundary
h = 0.25
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Xmesh = np.c_[xx.ravel(), yy.ravel()]
with torch.no_grad():
    scores = model(torch.tensor(Xmesh, dtype=torch.float32))
Z = (scores > 0).numpy().reshape(xx.shape)

fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())

## Q1.2: 

I use a constant learning rate of 1.0. See the effect of LR by exponentially decaying LR with each iteration reaching a value of 0.1 at the end of the iterations.

# Q2: NN Example 3 - Herschel Bulkley model

Modify the hyperparameters such as number of layers, neurons, regularization, learning rate, etc and evaluate its effect

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Generate data
num_samples = 10000
v_values = torch.rand(num_samples, 1) * 10
tau0_values = torch.rand(num_samples, 1) * 5
k_values = torch.rand(num_samples, 1) * 2
n_values = torch.rand(num_samples, 1) * 0.5 + 0.5 # 0.5 - 1.0

# Inputs
inputs = torch.cat([v_values, tau0_values, k_values, n_values], dim=1)

# Targets
y_true = tau0_values + k_values * (v_values**n_values)

# Model
model = nn.Sequential(
   nn.Linear(4, 64),
   nn.Tanh(),
   nn.Linear(64,1)
)

# Training
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

epochs = 5000
for epoch in range(epochs):
   y_pred = model(inputs)
   loss = loss_fn(y_pred, y_true)

   optimizer.zero_grad()
   loss.backward()
   optimizer.step()

# Evaluation
v_eval = torch.linspace(0, 10, 100).unsqueeze(1)
tau0_eval = 5 * torch.ones_like(v_eval)
k_eval = 2 * torch.ones_like(v_eval)
n_eval = 0.7 * torch.ones_like(v_eval)

inputs_eval = torch.cat([v_eval, tau0_eval, k_eval, n_eval], dim=1)
y_pred = model(inputs_eval)

y_true_eval = tau0_eval + k_eval * (v_eval**n_eval)

# Plot
plt.plot(v_eval.numpy(), y_true_eval.numpy(), 'r-', label='True')
plt.plot(v_eval.numpy(), y_pred.detach().numpy(), 'b.', label='Predicted')
plt.xlabel('Shear Rate')
plt.ylabel('Shear Stress')
plt.legend()
plt.show()